# Introduction to Computational science - Assignment 2
Sander Broos, Nick van Santen

In [ ]:
# Imports
from __future__ import annotations
from ipywidgets import *

import matplotlib.pyplot as plt
import numpy as np
import bisect
import math

from typing import Callable, List

In [ ]:
# Run cell to increase font sizes. Usefull when saving plots
SMALL_SIZE = 16
MEDIUM_SIZE = 20
BIGGER_SIZE = 24

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
class Event:

    def __init__(self, name: str, rate: Callable, event: dict):

        self.name = name
        self.event = event

        self.rate = rate

    def occur(self, groups, time):
        
        for group, value_to_add in self.event.items():
            groups[group].add_value(value_to_add, time)

class Simulator:

    def __init__(self, groups: List[Group], events: List[Event], max_time: float):

        self.groups = groups

        self.events = events

        self.time = 0
        self.max_time = max_time
        self.time_steps = [0]
    
    def update(self):

        total_rate = sum([event.rate(self.groups) for event in self.events])

        if total_rate == 0:
            self.time = self.max_time
            return

        r1 = np.random.rand()
        delta_time = -1 / total_rate * np.log(r1)
        self.time += delta_time

        if self.time > self.max_time:
            return

        self.time_steps.append(self.time)
        
        r2 = np.random.rand()
        P = r2 * total_rate
        event = self.determine_event(P)

        # Apply event
        event.occur(self.groups, self.time)

    def determine_event(self, p: float):
        
        value = 0
        for event in self.events:

            value += event.rate(self.groups)

            if value > p:
                return event
        
        print("ERROR: No event found")
        return None
    
    def total_n(self):
        return sum([group.number for group in self.groups.values()])
    
    def run(self):

        while self.time < self.max_time:
            self.update()

        for group in self.groups.values():

            group.append_to_history(group.number, self.max_time)

    def plot_group_levels(self):

        for group in self.groups.values():

            plt.plot(group.time_steps, group.history, label=group.name, drawstyle="steps-post")

        plt.legend()
        plt.show()

class Group:

    def __init__(self, name: str, initial: int):
        
        self.name = name
        self.history = [initial]
        self.time_steps = [0]

    @property
    def number(self):
        return self.history[-1]

    def add_value(self, value: int, time: float):

        self.append_to_history(self.number + value, time)

    def append_to_history(self, value: int, time: float):
        
        self.history.append(value)
        self.time_steps.append(time)


In [ ]:
class SIRSimulator(Simulator):

    def __init__(self, beta=3.0, gamma=1.0, delta=0.0, epsilon=0.0, mu=0.0, s_init=1000, i_init=5, r_init=0, max_time=10.0, **kwargs):

        groups = {
            "susceptible": Group("susceptible", s_init),
            "infected": Group("infected", i_init),
            "recovered": Group("recovered", r_init),
        }

        events = [
            Event(name="birth", 
                rate=lambda groups: mu * self.total_n(), 
                event={"susceptible": 1}),
            Event(name="transmission", 
                rate=lambda groups: beta * groups["susceptible"].number * groups["infected"].number / self.total_n(), 
                event={"susceptible": -1, "infected": 1}),
            Event(name="recovery", 
                rate=lambda groups: gamma * groups["infected"].number, 
                event={"infected": -1, "recovered": 1}),
            Event(name="death_s", 
                rate=lambda groups: mu * groups["susceptible"].number, 
                event={"susceptible": -1}),
            Event(name="death_i", 
                rate=lambda groups: mu * groups["infected"].number, 
                event={"infected": -1}),
            Event(name="death_r", 
                rate=lambda groups: mu * groups["recovered"].number, 
                event={"recovered": -1}),
            Event(name="import_move_in", 
                rate=lambda groups: delta * self.total_n()**0.5, 
                event={"infected": 1}),
            Event(name="import_pass_through", 
                rate=lambda groups: epsilon * groups["susceptible"].number * ((beta / (gamma + mu)) - 1) / self.total_n()**0.5, 
                event={"susceptible": -1, "infected": 1}),
        ]

        super().__init__(groups, events, max_time)
    

In [ ]:

def sir_gillespie(beta=3.0, gamma=1.0, delta=0.0, epsilon=0.0, mu=0.0, s_init=1000, i_init=5, r_init=0, max_time=10.0, show_plot=True):

    sim = SIRSimulator(**locals())
    sim.run()
    
    if show_plot:
        plt.clf()
        sim.plot_group_levels()

    return sim


In [ ]:
%matplotlib inline

interactive(sir_gillespie, beta=(0, 5.0), gamma=(0, 5.0), delta=(0, 5.0), epsilon=(0, 5.0), mu=(0, 1.0, 0.01), s_init=(0, 2000), i_init=(0, 1000), r_init=(0, 1000), max_time=(0, 1000), show_plot=True)

## Recreating figure 6.5

In [ ]:
N_values = range(100000, 1000000, 100000)
avg_values = []

for N in N_values:

    print(N)
    all_extinctions = []

    for _ in range(10):

      sim = sir_gillespie(beta=1.0, gamma=0.1, mu=5.5*10**-5, delta=0.02/365, s_init=N, i_init=0, max_time=365, show_plot=False)

      extinctions = 0
      infected_history = sim.groups["infected"].history

      for i in range(1, len(infected_history)):

        if infected_history[i] == 0 and infected_history[i-1] != 0:
            extinctions += 1
      
      all_extinctions.append(extinctions/10.0)
    
    avg_values.append(np.mean(all_extinctions))

plt.plot(N_values, avg_values)
plt.show()

## Variance

In [ ]:
%matplotlib inline
from statistics import variance

beta_values = np.linspace(0, 3, 50, endpoint=False)
variances = []

for beta in beta_values:
    
    end_infected = []
    print(round(beta, 8), end='--')

    for _ in range(200):
    
        sim = sir_gillespie(beta=beta, gamma=1.0, delta=0.0, epsilon=0.0, mu=0.0, s_init=1000, i_init=25, r_init=0, max_time=100.0, show_plot=False)
        end_infected.append(sim.groups["recovered"].number)

    variances.append(variance(end_infected))

plt.plot(beta_values, variances)
plt.show()

## Negative covariances

In [ ]:
def find_elem(elem, sorted_list):

    i = bisect.bisect_left(sorted_list, elem)

    if i != len(sorted_list) and sorted_list[i] == elem:
        return i

    return -1

def filter_common_times(groups, group1, group2):

    data1 = []
    data2 = []
    t_steps = groups[group1].time_steps

    for index1 in range(len(t_steps))[int(len(t_steps)/5):]:
        
        time1 = groups[group1].time_steps[index1]

        if find_elem(time1, groups[group2].time_steps) != -1:

            index2 = groups[group2].time_steps.index(time1)

            data1.append(groups[group1].history[index1])
            data2.append(groups[group2].history[index2])

    return data1, data2

In [ ]:
%matplotlib inline
sim = sir_gillespie(mu=0.2, max_time=100, show_plot=True)

s_data, i_data = filter_common_times(sim.groups, "susceptible", "infected")

print(f"Covariance: {np.cov(s_data, i_data)[0][1]}")
plt.plot(s_data, i_data)
plt.show()

In [66]:
N_values = np.linspace(100, 10000, 5, endpoint=False)
cov_averages = []

for N in N_values:
    print(N)

    covs = []
    i = 0

    while i < 5:

        sim = sir_gillespie(beta=15, gamma=1.216, mu=0.015, max_time=100, s_init=N, i_init=0, show_plot=False)
        s_data, i_data = filter_common_times(sim.groups, "susceptible", "infected")

        if sim.groups["infected"].history[-1] != 0:
            covs.append(np.cov(s_data, i_data)[0][1])
            i += 1

    cov_averages.append(np.mean(covs))

plt.plot(N_values, cov_averages)
plt.show

100.0


KeyboardInterrupt: 

## Extinction